# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import cities file into the cities_pd DataFrame
cities_pd = pd.read_csv("./Resources/cities.csv")
cities_pd.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,george town,20.0,MY,1.603852e+09,83.0,5.41,100.34,302.15,2.60
1,1,ushuaia,40.0,AR,1.603852e+09,42.0,-54.80,-68.30,280.15,6.20
2,2,isiro,100.0,CD,1.603852e+09,98.0,2.77,27.62,292.37,0.57
3,3,hilo,75.0,US,1.603852e+09,74.0,19.73,-155.09,300.15,1.50
4,4,barrow,75.0,US,1.603852e+09,68.0,71.29,-156.79,272.15,10.30


In [3]:
#check the data 
cities_pd.shape

(598, 10)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

In [5]:
#Drop na 
citiesna=cities_pd.dropna()

In [6]:
# Store latitude and longitude in locations
locations =citiesna[["Lat", "Lng"]]
#locations.shape

In [7]:
# Assign Humidity as the weight.
weight=citiesna["Humidity"]
#weight.shape

In [8]:
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=weight,dissipating=False, max_intensity=100,point_radius=3)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
#!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
* Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a * reasonable number.

In [10]:
#Some conditionsor a lot condition looks that my data is empty 
FittingWeather=cities_pd.loc[(cities_pd["Wind Speed"]< 10) & (cities_pd["Cloudiness"]==0)].dropna() 
#(cities_pd["Max Temp"]> 70) & (cities_pd["Max Temp"]< 80)].dropna()              

In [11]:
#Checking the Fitting Data 
FittingWeather.shape

(101, 10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#Store into variable named hotel_df
hotel_df=FittingWeather

In [13]:
#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

In [14]:
#Checking the final data with the conditions 
hotel_df.shape

(101, 11)

In [15]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates. #Store in data frame hote_info
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [16]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Results for  {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f" The closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing result ... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)

# Print end of search once searching is completed
print("-------End of Process-------")

Results for  8: punta arenas.
 The closest hotel in punta arenas is Hotel Cabo de Hornos.
------------
Results for  44: cabra.
 The closest hotel in cabra is MS Fuente Las Piedras.
------------
Results for  47: marzuq.
Missing result ... skipping.
------------
Results for  54: saint-philippe.
 The closest hotel in saint-philippe is Les Embruns Du Baril.
------------
Results for  60: svetlogorsk.
 The closest hotel in svetlogorsk is Hotel Svetlogorsk JBIK.
------------
Results for  62: konyshevka.
Missing result ... skipping.
------------
Results for  68: pisco.
 The closest hotel in pisco is Hotel Presidente.
------------
Results for  80: novouzensk.
Missing result ... skipping.
------------
Results for  93: yulara.
 The closest hotel in yulara is The Lost Camel Hotel - Ayers Rock Resort.
------------
Results for  102: krasnaya gora.
 The closest hotel in krasnaya gora is Sberbank Rossii.
------------
Results for  110: vocklabruck.
 The closest hotel in vocklabruck is Hotel Auerhahn.
-

 The closest hotel in guerrero negro is The Halfway Inn.
------------
Results for  553: birjand.
 The closest hotel in birjand is هتل پرويز.
------------
Results for  557: port lincoln.
 The closest hotel in port lincoln is Port Lincoln Hotel.
------------
Results for  558: rio gallegos.
 The closest hotel in rio gallegos is Hotel Aire de Patagonia.
------------
Results for  559: khasan.
Missing result ... skipping.
------------
Results for  561: vallenar.
 The closest hotel in vallenar is Hotel Esmeralda Limitada.
------------
Results for  570: chipinge.
 The closest hotel in chipinge is Chipinge Breeze Lodge.
------------
Results for  572: algeciras.
 The closest hotel in algeciras is H.PALMONES.
------------
Results for  579: tucson.
 The closest hotel in tucson is Lodge On The Desert.
------------
Results for  580: aden.
 The closest hotel in aden is Coral Hotel Aden.
------------
Results for  583: eregli.
 The closest hotel in eregli is CHERRY GARDEN CITY&SPA HOTEL.
------------
R

In [17]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))